### Chatbot And RAG Evaluation

Retrieval Augmented Generation (RAG) is a technique that enhances Large Language Models (LLMs) by providing them with relevant external knowledge. It has become one of the most widely used approaches for building LLM applications.

This tutorial will show you how to evaluate your RAG applications using LangSmith. You'll learn:

1. How to create test datasets
2. How to run your RAG application on those datasets
3. How to measure your application's performance using different evaluation metrics

#### Overview
A typical RAG evaluation workflow consists of three main steps:

1. Creating a dataset with questions and their expected answers
2. Running your RAG application on those questions
3. Using evaluators to measure how well your application performed, looking at factors like:
 - Answer relevance
 - Answer accuracy
 - Retrieval quality
 
For this tutorial, we'll create and evaluate a bot that answers questions about a few of Lilian Weng's insightful blog posts.

### Chatbot Evaluation

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_PROJECT"] = "RAG_EVALUATION"

#os.environ["LANGSMITH_API_KEY"]=os.getenv("LANGSMITH_API_KEY")
#os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
#os.environ["LANGSMITH_TRACING"]="true"

In [2]:
from langsmith import Client

client = Client()

# Define dataset: these are your test cases
dataset_name = "Chatbots Evaluation"
dataset = client.create_dataset(dataset_name)
client.create_examples(
    dataset_id=dataset.id,
    examples=[
        {
            "inputs": {"question": "What is LangChain?"},
            "outputs": {"answer": "A framework for building LLM applications"},
        },
        {
            "inputs": {"question": "What is LangSmith?"},
            "outputs": {"answer": "A platform for observing and evaluating LLM applications"},
        },
        {
            "inputs": {"question": "What is OpenAI?"},
            "outputs": {"answer": "A company that creates Large Language Models"},
        },
        {
            "inputs": {"question": "What is Google?"},
            "outputs": {"answer": "A technology company known for search"},
        },
        {
            "inputs": {"question": "What is Mistral?"},
            "outputs": {"answer": "A company that creates Large Language Models"},
        }
    ]
)

LangSmithConflictError: Conflict for /datasets. HTTPError('409 Client Error: Conflict for url: https://api.smith.langchain.com/datasets', '{"detail":"Dataset with this name already exists."}')

### Define Metrics (LLM As A Judge)


In [10]:

import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        #api_key=os.environ["GEMINI_API_KEY"]  # uses GOOGLE_API_KEY env var by default
    )

In [ ]:
# import openai
# from langsmith import wrappers
# from langchain.schema import SystemMessage, HumanMessage
# #openai_client=wrappers.wrap_openai(openai.OpenAI())

# eval_instructions = "You are an expert professor specialized in grading students' answers to questions."

# def correctness(inputs:dict,outputs:dict, reference_outputs:dict)->bool:
#       user_content = f"""You are grading the following question:
#     {inputs['question']}
#     Here is the real answer:
#     {reference_outputs['answer']}
#     You are grading the following predicted answer:
#     {outputs['response']}
#     Respond with CORRECT or INCORRECT:
#     Grade:
#     """
      
#       #response = llm.generate_content(f"[SYSTEM]: {eval_instructions}\n[USER]: {user_content}")
#       #content = response.text

#       # response=openai_client.chat.completions.create(
#       #       model="gpt-4o-mini",
#       #       temperature=0,
#       #       messages=[
#       #             {"role":"system","content":eval_instructions},
#       #             {"role":"user","content":user_content}
#       #       ]
#       # ).choices[0].message.content

# # Prepare the messages
# # Gemini uses a different message format - combine system and user messages
#       # Prepare messages in LangChain format
#       messages = [
#           SystemMessage(content=eval_instructions),
#           HumanMessage(content=user_content)
#       ]

#       # Generate response
#       response = llm.invoke(messages)

#       return response == "CORRECT"

In [11]:

import openai
from langsmith import wrappers
from langchain.schema import SystemMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

eval_instructions = "You are an expert professor specialized in grading students' answers to questions."

def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    user_content = f"""You are grading the following question:
{inputs['question']}

Here is the real answer:
{reference_outputs['answer']}

You are grading the following predicted answer:
{outputs['response']}

Respond with CORRECT or INCORRECT:
Grade:
"""
    
    # Prepare messages in LangChain format
    messages = [
        SystemMessage(content=eval_instructions),
        HumanMessage(content=user_content)
    ]
    
    try:
        # Generate response
        response = llm.invoke(messages)
        
        # Extract the text content from the response
        response_text = response.content.strip().upper()
        
        # Check if the response contains "CORRECT"
        # More robust checking for different response formats
        if "CORRECT" in response_text and "INCORRECT" not in response_text:
            return True
        elif "INCORRECT" in response_text:
            return False
        else:
            # If response is ambiguous, you might want to log it for debugging
            print(f"Ambiguous evaluator response: {response_text}")
            return False
            
    except Exception as e:
        print(f"Error in correctness evaluator: {e}")
        return False

# Alternative version with score-based approach (0.0 to 1.0)
def correctness_score(inputs: dict, outputs: dict, reference_outputs: dict) -> float:
    user_content = f"""You are grading the following question:
{inputs['question']}

Here is the real answer:
{reference_outputs['answer']}

You are grading the following predicted answer:
{outputs['response']}

Rate the correctness on a scale from 0 to 1, where:
- 0 = Completely incorrect
- 0.5 = Partially correct  
- 1 = Completely correct

Respond with only the numeric score (e.g., 0.8):
Score:
"""
    
    messages = [
        SystemMessage(content=eval_instructions),
        HumanMessage(content=user_content)
    ]
    
    try:
        response = llm.invoke(messages)
        response_text = response.content.strip()
        
        # Extract numeric score
        import re
        score_match = re.search(r'(\d+\.?\d*)', response_text)
        if score_match:
            score = float(score_match.group(1))
            # Ensure score is between 0 and 1
            return max(0.0, min(1.0, score))
        else:
            return 0.0
            
    except Exception as e:
        print(f"Error in correctness evaluator: {e}")
        return 0.0

# Updated concision evaluator for consistency
def concision(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    """Check if response is concise (shorter than 2x reference answer)"""
    try:
        response_length = len(outputs["response"])
        reference_length = len(reference_outputs["answer"])
        return response_length < 2 * reference_length
    except Exception as e:
        print(f"Error in concision evaluator: {e}")
        return False

In [ ]:
# ## Concisions- checks whether the actual output is less than 2x the length of the expected result.

# def concision(outputs: dict, reference_outputs: dict) -> bool:
#     return int(len(outputs["response"]) < 2 * len(reference_outputs["answer"]))

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import SystemMessage, HumanMessage

# Initialize model globally for better performance
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

default_instructions = "Respond to the users question in a short, concise manner (one short sentence)."

def my_app(question: str, model: str = "gemini-2.5-flash", instructions: str = default_instructions) -> str:
    """Main application function that generates responses using Gemini"""
    
    # For dynamic model switching, reinitialize if different model requested
    current_llm = llm
    if model != "gemini-2.5-flash":
        current_llm = ChatGoogleGenerativeAI(
            model=model,
            temperature=0,
            google_api_key=os.environ["GOOGLE_API_KEY"]
        )
    
    # Prepare messages
    messages = [
        SystemMessage(content=instructions),
        HumanMessage(content=question)
    ]
    
    # Generate and return response
    response = current_llm.invoke(messages)
    return response.content

def ls_target(inputs):
    """Target function for LangSmith evaluation"""
    
    # Extract question from inputs (adjust key based on your dataset)
    question = inputs.get("question", "") or inputs.get("input", "") or inputs.get("query", "")
    
    # You can customize model and instructions here if needed
    model = inputs.get("model", "gemini-2.5-flash")
    instructions = inputs.get("instructions", default_instructions)
    
    try:
        # Call your main app function
        response = my_app(
            question=question,
            model=model,
            instructions=instructions
        )
        
        # Return dictionary with string response for evaluators
        return {
            "response": response,
            "model_used": model
        }
        
    except Exception as e:
        # Handle errors gracefully
        return {
            "response": f"Error generating response: {str(e)}",
            "model_used": model
        }

# Run evaluation
experiment_results = client.evaluate(
    ls_target,  # Your target function
    data=dataset_name,  # Your dataset
    evaluators=[correctness, concision],  # Your evaluators
    experiment_prefix="gemini-2.5-flash-chatbot-new"
)

print(f"Experiment completed!")
print(f"Results: {experiment_results}")

View the evaluation results for experiment: 'gemini-2.5-flash-chatbot-new-4d4aee13' at:
https://smith.langchain.com/o/42698cb9-2205-48c5-886f-2174688c85d2/datasets/d31ff07c-2f7d-43c3-bde8-b65354cb695d/compare?selectedSessions=d15097bc-94ac-4c9f-9016-5919bd6db458




5it [00:34,  6.94s/it]

Experiment completed!
Results: <ExperimentResults gemini-2.5-flash-chatbot-new-4d4aee13>


### Run Evaluations

In [6]:
default_instructions = "Respond to the users question in a short, concise manner (one short sentence)."
def my_app(question: str, model: str = "gemini-2.5-flash", instructions: str = default_instructions) -> str:
    # return openai_client.chat.completions.create(
    #     model=model,
    #     temperature=0,
    #     messages=[
    #         {"role": "system", "content": instructions},
    #         {"role": "user", "content": question},
    #     ],
    # ).choices[0].message.content
 
    # Prepare the messages
    messages = [
            SystemMessage(content=instructions),
            HumanMessage(content=question)
        ]


        # Generate response
    response = llm.invoke(messages)
    return response

In [ ]:
# ### Call my_app for every datapoints
# def ls_target(inputs: str) -> dict:
#     return {"response": my_app(inputs["question"])}

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import SystemMessage, HumanMessage

def ls_target(inputs):
    """Target function that works with LangSmith evaluation"""
    
    # Initialize Gemini model
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        temperature=0,
        google_api_key="your-google-api-key"
    )
    
    # Extract question from inputs (adjust key name as needed)
    question = inputs.get("question", "") or inputs.get("input", "")
    
    # Prepare messages
    messages = [
        SystemMessage(content=eval_instructions),
        HumanMessage(content=question)
    ]
    
    try:
        # Generate response
        response = llm.invoke(messages)
        
        # Return dictionary with string content
        return {
            "response": response.content,  # String content for evaluators
            "model": "gemini-1.5-flash"   # Optional metadata
        }
    except Exception as e:
        return {
            "response": f"Error: {str(e)}",
            "model": "gemini-1.5-flash"
        }

# Run evaluation
experiment_results = client.evaluate(
    ls_target,
    data=dataset_name,
    evaluators=[correctness, concision],
    experiment_prefix="gemini-1.5-flash-chatbot"
)

print(f"Experiment results: {experiment_results}")

h:\akash\git\RAG-Techniques\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'gemini-1.5-flash-chatbot-7c73f549' at:
https://smith.langchain.com/o/42698cb9-2205-48c5-886f-2174688c85d2/datasets/d31ff07c-2f7d-43c3-bde8-b65354cb695d/compare?selectedSessions=2ec60a94-1aa3-4c32-ab35-93c09a3addea




5it [00:14,  3.00s/it]

Experiment results: <ExperimentResults gemini-1.5-flash-chatbot-7c73f549>


In [ ]:
# ## Run our evaluation
# experiment_results=client.evaluate(
#     ls_target, ## Your AI system
#     data=dataset_name,
#     evaluators=[correctness,concision],
#     experiment_prefix="gemini-2.5-flash-chatbot"
# )

View the evaluation results for experiment: 'gemini-2.5-flash-chatbot-a77c70db' at:
https://smith.langchain.com/o/42698cb9-2205-48c5-886f-2174688c85d2/datasets/d31ff07c-2f7d-43c3-bde8-b65354cb695d/compare?selectedSessions=670fb420-7585-4796-89a2-1edf86c564dc




0it [00:00, ?it/s]Error running evaluator <DynamicRunEvaluator correctness> on run b864d175-ff05-4cae-b11f-2b90c892956e: AttributeError("'ChatGoogleGenerativeAI' object has no attribute 'generate_content'")
Traceback (most recent call last):
  File "h:\akash\git\RAG-Techniques\venv\Lib\site-packages\langsmith\evaluation\_runner.py", line 1620, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(  # type: ignore[call-arg]
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "h:\akash\git\RAG-Techniques\venv\Lib\site-packages\langsmith\evaluation\evaluator.py", line 351, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "h:\akash\git\RAG-Techniques\venv\Lib\site-packages\langsmith\run_helpers.py", line 704, in wrapper
    function_result = run_container["context"].run(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "h:\akash\git\RAG-Techniques\venv\Lib\site-packages\langsmith\evaluation\evaluator.py", line 777, 

In [26]:
### Call my_app for every datapoints
def ls_target(inputs: str) -> dict:
    return {"response": my_app(inputs["question"],model="gemini-2.5")}

In [27]:
## Run our evaluation
experiment_results=client.evaluate(
    ls_target, ## Your AI system
    data=dataset_name,
    evaluators=[correctness,concision],
    experiment_prefix="gemini-2.5-chatbot"
)

View the evaluation results for experiment: 'gemini-2.5-chatbot-00bf92a1' at:
https://smith.langchain.com/o/42698cb9-2205-48c5-886f-2174688c85d2/datasets/d31ff07c-2f7d-43c3-bde8-b65354cb695d/compare?selectedSessions=9b7b391f-5f50-4966-a763-b337c21f712d




0it [00:00, ?it/s]Error running evaluator <DynamicRunEvaluator correctness> on run 54d7b89f-869a-4051-87e4-3064bc500398: AttributeError("'ChatGoogleGenerativeAI' object has no attribute 'generate_content'")
Traceback (most recent call last):
  File "h:\akash\git\RAG-Techniques\venv\Lib\site-packages\langsmith\evaluation\_runner.py", line 1620, in _run_evaluators
    evaluator_response = evaluator.evaluate_run(  # type: ignore[call-arg]
                         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "h:\akash\git\RAG-Techniques\venv\Lib\site-packages\langsmith\evaluation\evaluator.py", line 351, in evaluate_run
    result = self.func(
             ^^^^^^^^^^
  File "h:\akash\git\RAG-Techniques\venv\Lib\site-packages\langsmith\run_helpers.py", line 704, in wrapper
    function_result = run_container["context"].run(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "h:\akash\git\RAG-Techniques\venv\Lib\site-packages\langsmith\evaluation\evaluator.py", line 777, 

### Evaluation For RAG

In [15]:
## RAG
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# List of URLs to load documents from
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# Load documents from the URLs
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Initialize a text splitter with specified chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)

# Split the documents into chunks
doc_splits = text_splitter.split_documents(docs_list)

# Add the document chunks to the "vector store" using OpenAIEmbeddings
vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits,
    embedding=OpenAIEmbeddings(),
)

# With langchain we can easily turn any vector store into a retrieval component:
retriever = vectorstore.as_retriever(k=6)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [16]:
retriever.invoke("what is agents")

[Document(id='618dc4b1-b918-4844-ae2e-86af87f25256', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, 

Failed to multipart ingest runs: langsmith.utils.LangSmithRateLimitError: Rate limit exceeded for https://api.smith.langchain.com/runs/multipart. HTTPError('429 Client Error: Too Many Requests for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Too many requests: tenant exceeded usage limits: usage limit monthly_traces of 100 exceeded. Check LangSmith usage configuration settings"}\n')trace=2e2364fd-7c7c-4294-9669-f2c8acc71c76,id=2e2364fd-7c7c-4294-9669-f2c8acc71c76


In [18]:
from langchain.chat_models import init_chat_model
llm=init_chat_model("openai:gpt-4o-mini")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001EA28604B90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001EA91879FD0>, root_client=<openai.OpenAI object at 0x000001EA28604910>, root_async_client=<openai.AsyncOpenAI object at 0x000001EA286051D0>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [19]:
from langsmith import traceable

## Add decorator
@traceable()
def rag_bot(question:str)->dict:
    ## Relevant context
    docs=retriever.invoke(question)
    docs_string = " ".join(doc.page_content for doc in docs)

    instructions = f"""You are a helpful assistant who is good at analyzing source information and answering questions.       Use the following source documents to answer the user's questions.       If you don't know the answer, just say that you don't know.       Use three sentences maximum and keep the answer concise.

Documents:
{docs_string}"""
    
    ## llm invoke

    ai_msg=llm.invoke([
         {"role": "system", "content": instructions},
        {"role": "user", "content": question},

    ])
    return {"answer":ai_msg.content,"documents":docs}



In [20]:
rag_bot("What is agents")

{'answer': 'Agents refer to autonomous entities, particularly in the context of artificial intelligence, that can perceive their environment, make decisions, and take actions to achieve specific goals. In the documents, agents can be powered by large language models (LLMs) and possess features like memory, planning, and reflection to enhance their behavior and interaction. They are often designed to simulate human-like behavior or solve complex problems in various applications.',
 'documents': [Document(id='618dc4b1-b918-4844-ae2e-86af87f25256', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can b

### Dataset

In [21]:
from langsmith import Client

client=Client()

# Define the examples for the dataset
examples = [
    {
        "inputs": {"question": "How does the ReAct agent use self-reflection? "},
        "outputs": {"answer": "ReAct integrates reasoning and acting, performing actions - such tools like Wikipedia search API - and then observing / reasoning about the tool outputs."},
    },
    {
        "inputs": {"question": "What are the types of biases that can arise with few-shot prompting?"},
        "outputs": {"answer": "The biases that can arise with few-shot prompting include (1) Majority label bias, (2) Recency bias, and (3) Common token bias."},
    },
    {
        "inputs": {"question": "What are five types of adversarial attacks?"},
        "outputs": {"answer": "Five types of adversarial attacks are (1) Token manipulation, (2) Gradient based attack, (3) Jailbreak prompting, (4) Human red-teaming, (5) Model red-teaming."},
    }
]

### create the daatset and example in LAngsmith
dataset_name="RAG Test Evaluation"
dataset = client.create_dataset(dataset_name=dataset_name)
client.create_examples(
    dataset_id=dataset.id,
    examples=examples
)



{'example_ids': ['22098524-fe71-46c2-b86f-d0c8668808e6',
  '93bc3994-6e14-4bd8-b446-9a6c7eb712be',
  'd8baca57-c7a2-41a0-ae72-0d52b4cda043'],
 'count': 3}

### Evaluators or Metrics
1. Correctness: Response vs reference answer
- Goal: Measure "how similar/correct is the RAG chain answer, relative to a ground-truth answer"
- Mode: Requires a ground truth (reference) answer supplied through a dataset
- Evaluator: Use LLM-as-judge to assess answer correctness.

In [28]:
from typing_extensions import Annotated,TypedDict

## Correctness Output Schema

# Grade output schema
class CorrectnessGrade(TypedDict):
    # Note that the order in the fields are defined is the order in which the model will generate them.
    # It is useful to put explanations before responses because it forces the model to think through
    # its final response before generating it:
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    correct: Annotated[bool, ..., "True if the answer is correct, False otherwise."]

## correctness prompt

correctness_instructions = """You are a teacher grading a quiz. 

You will be given a QUESTION, the GROUND TRUTH (correct) ANSWER, and the STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Grade the student answers based ONLY on their factual accuracy relative to the ground truth answer. 
(2) Ensure that the student answer does not contain any conflicting statements.
(3) It is OK if the student answer contains more information than the ground truth answer, as long as it is factually accurate relative to the  ground truth answer.

Correctness:
A correctness value of True means that the student's answer meets all of the criteria.
A correctness value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

from langchain_openai import ChatOpenAI

grader_llm=ChatOpenAI(model="gpt-4o-mini",temperature=0).with_structured_output(CorrectnessGrade,
                                                                         method="json_schema",strict=True)
## evaluator
def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    """An evaluator for RAG answer accuracy"""
    answers = f"""\
QUESTION: {inputs['question']}
GROUND TRUTH ANSWER: {reference_outputs['answer']}
STUDENT ANSWER: {outputs['answer']}"""

    # Run evaluator
    grade = grader_llm.invoke([
        {"role": "system", "content": correctness_instructions}, 
        {"role": "user", "content": answers}
    ])
    return grade["correct"]







### Relevance: Response vs input
The flow is similar to above, but we simply look at the inputs and outputs without needing the reference_outputs. Without a reference answer we can't grade accuracy, but can still grade relevance—as in, did the model address the user's question or not.

In [24]:
# Grade output schema
class RelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "Provide the score on whether the answer addresses the question"]

# Grade prompt
relevance_instructions="""You are a teacher grading a quiz. 

You will be given a QUESTION and a STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is concise and relevant to the QUESTION
(2) Ensure the STUDENT ANSWER helps to answer the QUESTION

Relevance:
A relevance value of True means that the student's answer meets all of the criteria.
A relevance value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader LLM
relevance_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(RelevanceGrade, method="json_schema", strict=True)

# Evaluator
def relevance(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer helpfulness."""
    answer = f"QUESTION: {inputs['question']}\nSTUDENT ANSWER: {outputs['answer']}"
    grade = relevance_llm.invoke([
        {"role": "system", "content": relevance_instructions}, 
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

### Groundedness: Response vs retrieved docs
Another useful way to evaluate responses without needing reference answers is to check if the response is justified by (or "grounded in") the retrieved documents.

In [25]:
# Grade output schema
class GroundedGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    grounded: Annotated[bool, ..., "Provide the score on if the answer hallucinates from the documents"]

# Grade prompt
grounded_instructions = """You are a teacher grading a quiz. 

You will be given FACTS and a STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Ensure the STUDENT ANSWER is grounded in the FACTS. 
(2) Ensure the STUDENT ANSWER does not contain "hallucinated" information outside the scope of the FACTS.

Grounded:
A grounded value of True means that the student's answer meets all of the criteria.
A grounded value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader LLM 
grounded_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(GroundedGrade, method="json_schema", strict=True)

# Evaluator
def groundedness(inputs: dict, outputs: dict) -> bool:
    """A simple evaluator for RAG answer groundedness."""
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nSTUDENT ANSWER: {outputs['answer']}"
    grade = grounded_llm.invoke([{"role": "system", "content": grounded_instructions}, {"role": "user", "content": answer}])
    return grade["grounded"]

### Retrieval Relevance: Retrieved docs vs input

In [26]:
# Grade output schema
class RetrievalRelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "True if the retrieved documents are relevant to the question, False otherwise"]

# Grade prompt
retrieval_relevance_instructions = """You are a teacher grading a quiz. 

You will be given a QUESTION and a set of FACTS provided by the student. 

Here is the grade criteria to follow:
(1) You goal is to identify FACTS that are completely unrelated to the QUESTION
(2) If the facts contain ANY keywords or semantic meaning related to the question, consider them relevant
(3) It is OK if the facts have SOME information that is unrelated to the question as long as (2) is met

Relevance:
A relevance value of True means that the FACTS contain ANY keywords or semantic meaning related to the QUESTION and are therefore relevant.
A relevance value of False means that the FACTS are completely unrelated to the QUESTION.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader LLM
retrieval_relevance_llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(RetrievalRelevanceGrade, method="json_schema", strict=True)

def retrieval_relevance(inputs: dict, outputs: dict) -> bool:
    """An evaluator for document relevance"""
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    answer = f"FACTS: {doc_string}\nQUESTION: {inputs['question']}"

    # Run evaluator
    grade = retrieval_relevance_llm.invoke([
        {"role": "system", "content": retrieval_relevance_instructions}, 
        {"role": "user", "content": answer}
    ])
    return grade["relevant"]

### Run the evaluation

In [34]:
def target(inputs: dict) -> dict:
    return rag_bot(inputs["question"])

experiment_results = client.evaluate(
    target,
    data=dataset_name,
    evaluators=[correctness, groundedness, relevance, retrieval_relevance],
    experiment_prefix="rag-doc-relevance",
    metadata={"version": "LCEL context, gpt-4-0125-preview"},
)
# Explore results locally as a dataframe if you have pandas installed
experiment_results.to_pandas()

View the evaluation results for experiment: 'rag-doc-relevance-aea19ef6' at:
https://smith.langchain.com/o/df51e0aa-baf3-5e22-92d8-ad1de74986e0/datasets/f7dc8c23-3b36-46c2-ba44-9f7d6dc50529/compare?selectedSessions=28f937b5-5274-40e6-b777-276ac59842a6




3it [00:40, 13.64s/it]


,inputs.question,outputs.answer,outputs.documents,error,reference.answer,feedback.correctness,feedback.groundedness,feedback.relevance,feedback.retrieval_relevance,execution_time,example_id,id
0,How does the ReAct agent use self-reflection?,The ReAct agent utilizes self-reflection by it...,[page_content='Self-reflection is a vital aspe...,None,"ReAct integrates reasoning and acting, perform...",True,False,True,False,2.469538,22098524-fe71-46c2-b86f-d0c8668808e6,b6c2d547-afdd-4cdc-9691-2082d32928e7
1,What are the types of biases that can arise wi...,The types of biases that can arise with few-sh...,[page_content='Zero-shot and few-shot learning...,None,The biases that can arise with few-shot prompt...,True,True,True,True,3.005110,93bc3994-6e14-4bd8-b446-9a6c7eb712be,ff2bf2ae-213d-486a-a85d-e0fea4912846
2,What are five types of adversarial attacks?,The five types of adversarial attacks are toke...,[page_content='Black-box attacks assume that a...,None,Five types of adversarial attacks are (1) Toke...,True,True,True,True,1.783208,d8baca57-c7a2-41a0-ae72-0d52b4cda043,41383259-c822-4859-853b-ca1dd629f6cf
